## Import Tools

In [1]:
import torch
import torch.nn as nn
import numpy as np
import scipy.io 
import random
import math
import matplotlib.pyplot as plt
import torch.nn.functional as F
import os
import seaborn as sn
import pandas as pd
from inspect import getsource
os.environ['KMP_DUPLICATE_LIB_OK']='True' 

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

## Dataset Processing 

### Read in the original dataset 

In [43]:
train_dl_origin = torch.load('Dataset/train_dl.pt')
valid_dl_origin = torch.load('Dataset/valid_dl.pt')

train_CSI = train_dl_origin.dataset[:][0]
train_label = train_dl_origin.dataset[:][1][:,2].type(torch.LongTensor)

valid_CSI = valid_dl_origin.dataset[:][0]
valid_label = valid_dl_origin.dataset[:][1][:,2].type(torch.LongTensor)

### CSI Processing: Take Modulus of complex matrices

In [44]:
train_CSI_modulus = torch.abs(train_CSI)
valid_CSI_modulus = torch.abs(valid_CSI)

In [45]:
print(train_CSI_modulus.shape)
print(valid_CSI_modulus.shape)

torch.Size([15000, 1, 4, 1632])
torch.Size([5000, 1, 4, 1632])


###  CSI Processing: Normalize to [0,1]

In [46]:
#Normalize the data to [0,1]
train_CSI_modulus = train_CSI_modulus/torch.max(train_CSI_modulus)
valid_CSI_modulus = valid_CSI_modulus/torch.max(valid_CSI_modulus)

#Make the input data 2D
train_CSI_modulus_2D = train_CSI_modulus.reshape(train_CSI_modulus.shape[0],-1)
valid_CSI_modulus_2D = valid_CSI_modulus.reshape(valid_CSI_modulus.shape[0],-1)

print(train_CSI_modulus.shape)
print(valid_CSI_modulus.shape)
print(train_label.shape)

torch.Size([15000, 1, 4, 1632])
torch.Size([5000, 1, 4, 1632])
torch.Size([15000])


### CSI to Logistic Regression

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score
import time
rstate = 23

# create a logistic regression model
logreg = LogisticRegression(random_state=rstate, max_iter=1000)

# fit the model to the training data
logreg.fit(train_CSI_modulus.reshape(train_CSI_modulus.shape[0], -1), train_label)

#test training accuracy and test accuracy
start_time = time.time()
train_pred = logreg.predict(train_CSI_modulus.reshape(train_CSI_modulus.shape[0], -1))
train_acc = accuracy_score(train_label, train_pred)
train_precision = precision_score(train_label, train_pred)
train_recall = recall_score(train_label, train_pred)
train_f1 = f1_score(train_label, train_pred)
end_time = time.time()
print('Training Accuracy: ', train_acc)
print('Training Precision: ', train_precision)
print('Training Recall: ', train_recall)
print('Training F1 Score: ', train_f1)
print('Training Time: ', end_time-start_time)

start_valid_time = time.time()
valid_pred = logreg.predict(valid_CSI_modulus.reshape(valid_CSI_modulus.shape[0], -1))
valid_acc = accuracy_score(valid_label, valid_pred)
valid_precision = precision_score(valid_label, valid_pred)
valid_recall = recall_score(valid_label, valid_pred)
valid_f1 = f1_score(valid_label, valid_pred)
end_valid_time = time.time()
print('Validation Accuracy: ', valid_acc)
print('Validation Precision: ', valid_precision)
print('Validation Recall: ', valid_recall)
print('Validation F1 Score: ', valid_f1)
print('Validation Time: ', end_valid_time-start_valid_time)



Training Accuracy:  0.9598666666666666
Training Precision:  0.9951876804619827
Training Recall:  0.7774436090225564
Training F1 Score:  0.8729421696918531
Training Time:  0.6648011207580566
Validation Accuracy:  0.9474
Validation Precision:  0.9869753979739508
Validation Recall:  0.7286324786324786
Validation F1 Score:  0.8383527965580824
Validation Time:  0.48642992973327637


### CSI to Sentiment Classifier Converted to Binary Classifier

In [67]:
import time
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
# define the neural network model
class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(26112, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        #print('shape 0 ' + str(x.shape))
        x = self.pool(F.relu(self.conv1(x)))
        #print('shape 1 ' + str(x.shape))
        x = self.pool(F.relu(self.conv2(x)))
        #print('shape 2 ' + str(x.shape))
        x = x.view(x.size(0), -1)
        #print('shape 3 ' + str(x.shape))
        x = F.relu(self.fc1(x))
        #print('shape 4 ' + str(x.shape))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.sigmoid(x).squeeze(1)
        #print('shape 5 ' + str(x.shape))
        return x

def psource(*functions):
    """Print the source code for the given function(s)."""
    source_code = '\n\n'.join(getsource(fn) for fn in functions)
    print(source_code)



# create the dataset and dataloader
train_dataset = TensorDataset(train_CSI_modulus, train_label)
valid_dataset = TensorDataset(valid_CSI_modulus, valid_label)

print(train_CSI_modulus.shape)
print(valid_CSI_modulus.shape)

train_dl = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dl = DataLoader(valid_dataset, batch_size=32, shuffle=False)

# create the model and optimizer
model = BinaryClassifier()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

criterion = nn.BCEWithLogitsLoss()




# train the model
num_epochs = 1
start_time = time.time()
for epoch in range(num_epochs):
    epoch_start_time = time.time()
    for i, (inputs, labels) in enumerate(train_dl):
        if i%40==0 or i==len(train_dl):
            print('shape set ' + str(i) + ' of ' + str(len(train_dl)) + ' ' + str(inputs.shape))
        optimizer.zero_grad()
        outputs = model(inputs)
        print('outputs shape ' + str(outputs.shape))
        print('outputs ' + str(outputs))
        #loss = F.binary_cross_entropy_with_logits(outputs, labels.float())
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()

    epoch_end_time = time.time()
    epoch_time = epoch_end_time - epoch_start_time
    print('Epoch: [{}/{}], Time: {:.4f}s'.format(epoch+1, num_epochs, epoch_time))

end_time = time.time()
total_time = end_time - start_time
print('Training Time: {:.4f}s'.format(total_time))

start_test_time = time.time()
# set the model to evaluation mode
model.eval()

valid_preds = []
valid_labels = []
with torch.no_grad():
    for inputs, labels in valid_dl:
        outputs = model(inputs)
        preds = outputs.round()
        valid_preds.extend(preds.tolist())
        valid_labels.extend(labels.tolist())

#cm = confusion_matrix(valid_labels, valid_preds)
#print(cm)
#print(classification_report(valid_labels, valid_preds))
#for i, label in enumerate(valid_labels):
#    print("Label: ", label, " Prediction: ", valid_preds[i])

# calculate the confusion matrix
cm = confusion_matrix(valid_labels, valid_preds)

# calculate the metrics
tn, fp, fn, tp = cm.ravel()
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)
end_test_time = time.time()

print('Confusion Matrix:')
print(cm)
print('Accuracy: {:.4f}'.format(accuracy))
print('Precision: {:.4f}'.format(precision))
print('Recall: {:.4f}'.format(recall))
print('F1 Score: {:.4f}'.format(f1))
test_time = end_test_time - start_test_time
print('Test Time: {:.4f}s'.format(test_time))




torch.Size([15000, 1, 4, 1632])
torch.Size([5000, 1, 4, 1632])
shape set 0 of 469 torch.Size([32, 1, 4, 1632])


RuntimeError: result type Float can't be cast to the desired output type Long

In [61]:
start_test_time = time.time()
# set the model to evaluation mode
model.eval()

valid_preds = []
valid_labels = []
with torch.no_grad():
    for inputs, labels in valid_dl:
        outputs = model(inputs)
        print(outputs)
        preds = outputs.round()
        valid_preds.extend(preds.tolist())
        valid_labels.extend(labels.tolist())

#cm = confusion_matrix(valid_labels, valid_preds)
#print(cm)
#print(classification_report(valid_labels, valid_preds))
#for i, label in enumerate(valid_labels):
#    print("Label: ", label, " Prediction: ", valid_preds[i])

# calculate the confusion matrix
cm = confusion_matrix(valid_labels, valid_preds)

# calculate the metrics
tn, fp, fn, tp = cm.ravel()
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * precision * recall / (precision + recall)
end_test_time = time.time()

print('Confusion Matrix:')
print(cm)
print('Accuracy: {:.4f}'.format(accuracy))
print('Precision: {:.4f}'.format(precision))
print('Recall: {:.4f}'.format(recall))
print('F1 Score: {:.4f}'.format(f1))
test_time = end_test_time - start_test_time
print('Test Time: {:.4f}s'.format(test_time))

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.

/var/folders/9l/blmtv3cs5xsdjhpsftz2yxl80000gn/T/ipykernel_27619/1243102087.py:27: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)


In [14]:
import time
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(26112, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = self.sigmoid(x).squeeze(1)
        return x

model = BinaryClassifier()
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_dataset = TensorDataset(train_CSI_modulus, train_label)
valid_dataset = TensorDataset(valid_CSI_modulus, valid_label)

print(train_CSI_modulus.shape)
print(valid_CSI_modulus.shape)

train_dl = DataLoader(train_dataset, batch_size=200, shuffle=True)
valid_dl = DataLoader(valid_dataset, batch_size=200, shuffle=False)

num_epochs = 1
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_dl):
        print('shape set ' + str(i) + ' of ' + str(len(train_dl)) + ' ' + str(inputs.shape))
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.float())
        loss.backward()
        optimizer.step()
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))
    
    # set the model to evaluation mode
    model.eval()

    valid_preds = []
    valid_labels = []
    with torch.no_grad():
        for inputs, labels in valid_dl:
            outputs = model(inputs)
            preds = outputs.round()
            valid_preds.extend(preds.tolist())
            valid_labels.extend(labels.tolist())

    # calculate the confusion matrix
    cm = confusion_matrix(valid_labels, valid_preds)

    # calculate the metrics
    tn, fp, fn, tp = cm.ravel()
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)

    print('Confusion Matrix:')
    print(cm)
    print('Accuracy: {:.4f}'.format(accuracy))
    print('Precision: {:.4f}'.format(precision))
    print('Recall: {:.4f}'.format(recall))
    print('F1 Score: {:.4f}'.format(f1))



torch.Size([15000, 1, 4, 1632])
torch.Size([5000, 1, 4, 1632])
shape set 0 of 75 torch.Size([200, 1, 4, 1632])
shape set 1 of 75 torch.Size([200, 1, 4, 1632])
shape set 2 of 75 torch.Size([200, 1, 4, 1632])
shape set 3 of 75 torch.Size([200, 1, 4, 1632])
shape set 4 of 75 torch.Size([200, 1, 4, 1632])
shape set 5 of 75 torch.Size([200, 1, 4, 1632])
shape set 6 of 75 torch.Size([200, 1, 4, 1632])
shape set 7 of 75 torch.Size([200, 1, 4, 1632])
shape set 8 of 75 torch.Size([200, 1, 4, 1632])
shape set 9 of 75 torch.Size([200, 1, 4, 1632])
shape set 10 of 75 torch.Size([200, 1, 4, 1632])
shape set 11 of 75 torch.Size([200, 1, 4, 1632])
shape set 12 of 75 torch.Size([200, 1, 4, 1632])
shape set 13 of 75 torch.Size([200, 1, 4, 1632])
shape set 14 of 75 torch.Size([200, 1, 4, 1632])
shape set 15 of 75 torch.Size([200, 1, 4, 1632])
shape set 16 of 75 torch.Size([200, 1, 4, 1632])
shape set 17 of 75 torch.Size([200, 1, 4, 1632])
shape set 18 of 75 torch.Size([200, 1, 4, 1632])
shape set 19 of 

NameError: name 'confusion_matrix' is not defined

In [33]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report

class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        self.layer_1 = nn.Linear(1632, 64)
        self.layer_2 = nn.Linear(1632, 64)
        self.layer_out = nn.Linear(64, 1) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(64)
        self.batchnorm2 = nn.BatchNorm1d(64)
        
    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)
        
        return x

train_dataset = TensorDataset(train_CSI_modulus, train_label)
valid_dataset = TensorDataset(valid_CSI_modulus, valid_label)

print(train_CSI_modulus.shape)
print(valid_CSI_modulus.shape)

train_dl = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dl = DataLoader(valid_dataset, batch_size=32, shuffle=False)

model = BinaryClassification()
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)
    
    return acc

num_epochs = 1

model.train()
for e in range(1, num_epochs+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in train_dl:
        optimizer.zero_grad()
        
        y_pred = model(X_batch)
        
        loss = criterion(y_pred, y_batch.unsqueeze(1))
        acc = binary_acc(y_pred, y_batch.unsqueeze(1))
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        

    print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_dl):.5f} | Acc: {epoch_acc/len(train_dl):.3f}')


torch.Size([15000, 1, 4, 1632])
torch.Size([5000, 1, 4, 1632])


ValueError: expected 2D or 3D input (got 4D input)

### CSI to Binary Classifier v2

In [56]:
from sklearn.metrics import confusion_matrix
# Convert numpy arrays to PyTorch tensors
X_train = torch.FloatTensor(train_CSI_modulus_2D)
y_train = torch.LongTensor(train_label)  # Assuming binary classification

X_test = torch.FloatTensor(valid_CSI_modulus_2D)
y_test = torch.LongTensor(valid_label)  # Assuming binary classification



class FNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Hyperparameters
print("shape of X_train",X_train.shape)
input_size = X_train.shape[1]
hidden_size = 64
output_size = 2  # Binary classification (LOS/NLOS)
learning_rate = 0.1
num_epochs = 10

# Instantiate the model, loss function, and optimizer
model = FNNModel(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Testing the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted = torch.max(test_outputs, 1)  # Take the class with the maximum probability


    # Calculate the confusion matrix
    cm = confusion_matrix(y_test, predicted)

    # Print the confusion matrix
    print('Confusion Matrix:')
    print(cm)

    # Calculate the accuracy, precision, and recall
    tn, fp, fn, tp = cm.ravel()
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)

    # Print the accuracy, precision, and recall
    print('Accuracy: {:.4f}'.format(accuracy))
    print('Precision: {:.4f}'.format(precision))
    print('Recall: {:.4f}'.format(recall))
    print('F1 Score: {:.4f}'.format(f1))

# Evaluate accuracy
accuracy = (predicted == y_test).sum().item() / y_test.size(0)
print(f'Accuracy: {accuracy:.4f}')

shape of X_train torch.Size([15000, 6528])
Epoch [1/10], Loss: 0.6909
Epoch [2/10], Loss: 61.7929
Epoch [3/10], Loss: 17.4500
Epoch [4/10], Loss: 79.6815
Epoch [5/10], Loss: 1.4679
Epoch [6/10], Loss: 7.4972
Epoch [7/10], Loss: 9.4113
Epoch [8/10], Loss: 7.1829
Epoch [9/10], Loss: 1.0333
Epoch [10/10], Loss: 56.8914
Confusion Matrix:
[[3356  708]
 [ 593  343]]
Accuracy: 0.7398
Precision: 0.3264
Recall: 0.3665
Accuracy: 0.7398


In [57]:
from sklearn.metrics import confusion_matrix
import time
# Convert numpy arrays to PyTorch tensors
X_train = torch.FloatTensor(train_CSI_modulus_2D)
y_train = torch.LongTensor(train_label)  # Assuming binary classification

X_test = torch.FloatTensor(valid_CSI_modulus_2D)
y_test = torch.LongTensor(valid_label)  # Assuming binary classification



class FNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Hyperparameters
print("shape of X_train",X_train.shape)
input_size = X_train.shape[1]
hidden_size = 64
output_size = 2  # Binary classification (LOS/NLOS)
learning_rate = 0.1
num_epochs = 10

# Instantiate the model, loss function, and optimizer
model = FNNModel(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
start_train_time = time.time()
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
end_train_time = time.time()
train_time = end_train_time - start_train_time
print('Training Time: {:.4f}s'.format(train_time))

# Testing the model
start_test_time = time.time()
model.eval()
with torch.no_grad():
    test_outputs = model(X_test)
    _, predicted = torch.max(test_outputs, 1)  # Take the class with the maximum probability


    # Calculate the confusion matrix
    cm = confusion_matrix(y_test, predicted)

    # Print the confusion matrix
    print('Confusion Matrix:')
    print(cm)

    # Calculate the accuracy, precision, and recall
    tn, fp, fn, tp = cm.ravel()
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)

    # Print the accuracy, precision, and recall
    print('Accuracy: {:.4f}'.format(accuracy))
    print('Precision: {:.4f}'.format(precision))
    print('Recall: {:.4f}'.format(recall))
    print('F1 Score: {:.4f}'.format(f1))
end_test_time = time.time()
test_time = end_test_time - start_test_time
print('Test Time: {:.4f}s'.format(test_time))

# Evaluate accuracy
accuracy = (predicted == y_test).sum().item() / y_test.size(0)
print(f'Accuracy: {accuracy:.4f}')

shape of X_train torch.Size([15000, 6528])
Epoch [1/10], Loss: 0.7118
Epoch [2/10], Loss: 36.5811
Epoch [3/10], Loss: 120.9331
Epoch [4/10], Loss: 36.9671
Epoch [5/10], Loss: 73.4423
Epoch [6/10], Loss: 89.7117
Epoch [7/10], Loss: 88.1263
Epoch [8/10], Loss: 74.1237
Epoch [9/10], Loss: 53.7189
Epoch [10/10], Loss: 32.0828
Training Time: 5.5409s
Confusion Matrix:
[[4064    0]
 [ 936    0]]
Accuracy: 0.8128
Precision: nan
Recall: 0.0000
F1 Score: nan
Test Time: 0.1521s
Accuracy: 0.8128


/var/folders/9l/blmtv3cs5xsdjhpsftz2yxl80000gn/T/ipykernel_2959/4055686273.py:71: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)
